In [1]:
# -----------------------------
# 1. Import SparkSession
# -----------------------------
from pyspark.sql import SparkSession
import time

# -----------------------------
# 2. Create SparkSession with Hive support
# -----------------------------

gluten_jar = "/opt/gluten/gluten-velox-bundle-spark3.5_2.12-centos_7_x86_64-1.4.0.jar"

spark = (
    SparkSession.builder
        .config("spark.plugins", "org.apache.gluten.GlutenPlugin") 
        .config("spark.memory.offHeap.size", "8g")  
        .config("spark.shuffle.manager","org.apache.spark.shuffle.sort.ColumnarShuffleManager")  
        .config("spark.memory.offHeap.enabled", "true") 
        .config("spark.driver.extraClassPath",gluten_jar)
        .config("spark.executor.extraClassPath",gluten_jar)
        .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
        .config("spark.hadoop.fs.s3a.bucket.all.committer.magic.enabled", "true")
        .config("spark.eventLog.enabled", "false")
        .config("spark.dynamicAllocation.enabled", "false")
        .config("spark.log.level", "warn")
        .config("spark.sql.catalogImplementation", "hive")
        .config("hive.metastore.uris", "thrift://hive-metastore:9083")
        .enableHiveSupport()
        .getOrCreate()
)

print("Spark Session Created!")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 08:55:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".
25/11/24 08:55:01 ERROR SparkContext: Error initializing SparkContext.
java.lang.ClassNotFoundException: org.apache.spark.shuffle.sort.ColumnarShuffleManager
	at java.base/java.net.URLClassLoader.findClass(Unknown Source)
	at java.base/java.lang.ClassLoader.loadClass(Unknown Source)
	at java.base/java.lang.ClassLoader.loadClass(Unknown Source)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Unknown Source)
	at org.apache.spark.util.SparkClassUtils.classForName(SparkClassUtils.scala:41)
	at org.apache.spark.util.SparkClassUtils.classForName$(SparkClassUtils.scala:36)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:94)
	at org.apach

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.ClassNotFoundException: org.apache.spark.shuffle.sort.ColumnarShuffleManager
	at java.base/java.net.URLClassLoader.findClass(Unknown Source)
	at java.base/java.lang.ClassLoader.loadClass(Unknown Source)
	at java.base/java.lang.ClassLoader.loadClass(Unknown Source)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Unknown Source)
	at org.apache.spark.util.SparkClassUtils.classForName(SparkClassUtils.scala:41)
	at org.apache.spark.util.SparkClassUtils.classForName$(SparkClassUtils.scala:36)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:94)
	at org.apache.spark.util.Utils$.instantiateSerializerOrShuffleManager(Utils.scala:2548)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:323)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:196)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:284)
	at org

In [3]:
# List databases
# spark.sql("""
# CREATE DATABASE tpch100gb
# LOCATION 's3a://akashx-use1-s3-input-bucket/camline/tpch100gb';
# """)
spark.sql("""
use DATABASE tpch100gb; """)
tables = spark.sql("SHOW tables")
tables.show(truncate=False)


UnsupportedOperationException: empty.reduceLeft

In [85]:
spark.sql("""
use DATABASE tpch100gb;
""")
sql_text = open("demo.sql").read()
for statement in sql_text.split(";"):
    stmt = statement.strip()
    if stmt:
        start = time.time()
        print("Running:", stmt[:80], "...")
        spark.sql(stmt)
        end = time.time()
        duration = end - start
        print(f"Time taken: {duration:.3f} seconds")


Running: USE tpch100gb ...
Time taken: 0.005 seconds
Running: DROP TABLE IF EXISTS tpch100gb.supplier6 ...
Time taken: 0.006 seconds
Running: CREATE TABLE tpch100gb.supplier6
USING DELTA
LOCATION 's3a://akashx-use1-s3-inpu ...


25/11/19 14:21:53 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`tpch100gb`.`supplier6` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


Time taken: 4.973 seconds


In [86]:
# print the count of the table
tbl = spark.sql("select count(*) from supplier5 limit 1 ")
tbl.show(truncate=False)

+--------+
|count(1)|
+--------+
|1000000 |
+--------+

